In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
%matplotlib inline
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:
#Create a sqlite3 connection to the database
connex = sqlite3.connect('database.sqlite')
cur = connex.cursor()

In [58]:
#Pull all data out of the database into a DataFrame
loanbook = pd.read_sql_query('select * from loan;', connex)

In [4]:
len(loanbook.columns)
#Number of columns

75

In [5]:
lbk_desc = loanbook.describe()
lbk_desc.ix['count'].sort_values()

dti_joint                         509.0
annual_inc_joint                  511.0
il_util                         18617.0
mths_since_rcnt_il              20810.0
all_util                        21372.0
total_cu_tl                     21372.0
open_acc_6m                     21372.0
open_il_6m                      21372.0
open_il_12m                     21372.0
open_il_24m                     21372.0
inq_fi                          21372.0
total_bal_il                    21372.0
inq_last_12m                    21372.0
max_bal_bc                      21372.0
open_rv_12m                     21372.0
open_rv_24m                     21372.0
mths_since_last_record         137053.0
mths_since_last_major_derog    221704.0
mths_since_last_delinq         433069.0
total_rev_hi_lim               817106.0
tot_cur_bal                    817106.0
tot_coll_amt                   817106.0
collections_12_mths_ex_med     887237.0
acc_now_delinq                 887353.0
open_acc                       887353.0


In [112]:
dropcols = lbk_desc.ix['count'] < (lbk_desc.ix['count'].max() *.8)
dropcols = dropcols[dropcols==True]
len(dropcols)

19

In [59]:
loanbook_full = loanbook.drop(dropcols.index, axis = 1)

In [67]:
uniqVal = loanbook_full.apply(lambda x: x.nunique(), axis=0)

In [111]:
uniqVals_5_Greater = uniqVal[uniqVal>5]
len(uniqVals_5_Greater)

49

In [74]:
loanbook_full = loanbook_full[uniqVals_5_Greater.index]

In [75]:
#What are the unique values of the loan statuses? 
loanbook_full['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)', None,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [76]:
#This is a how the values for a good loan will be mapped off of the existing loan_status
d = {'Current': True, 'Fully Paid':True, 'Charged Off': False, 'Default':False, 'Late (31-120 days)':False, 'In Grace Period':False, 'Late (16-30 days)':False, 'Does not meet the credit policy. Status:Fully Paid':True, 'Does not meet the credit policy. Status:Charged Off':False, None:False, 'Issued':True }

In [81]:
#New Column created for a good loan status based off of the mapping 
loanbook_full.loc['GoodLoan'] = loanbook['loan_status'].map(d)
loanbook_full['GoodLoan'].head()

0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
Name: GoodLoan, dtype: float64

In [82]:
loanbook['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE', None, 'ANY'], dtype=object)

In [83]:
dd = {'OWN':'OWN', 'NONE':'OTHER', 'OTHER':'OTHER', 'MORTGAGE':'OWN', None:'OTHER', 'ANY':'OTHER', 'NONE':'OTHER'}

In [84]:
loanbook_full.loc['home_ownership'] = loanbook['home_ownership'].map(dd)

In [85]:
loanbooksum = loanbook_full.describe()
#loanbooksum.to_csv('loanbookFeaturesInt.csv', ',')

In [15]:
loanbook_clean = loanbook[newColNames]

In [86]:
loanbook['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a', None], dtype=object)

In [87]:
ddd = {'10+ years':10, '< 1 year':0, '1 year':3, '2 years':2, '3 years':3, '4 years':4, '5 years':5, '6 years':6, '7 years':7, '8 years':8, '9 years':9, 'n/a':0, None:0}

In [88]:
loanbook_full.loc['emp_length'] = loanbook_clean['emp_length'].map(ddd)

In [89]:
loanbook_full.dtypes

index                         float64
id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
issue_d                        object
loan_status                    object
url                            object
desc                           object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

In [116]:
newColNames=['member_id', 'loan_amnt', 'int_rate', 'installment', 'emp_length', 'home_ownership', 'annual_inc', 'issue_d', 'loan_status', 'zip_code', 'addr_state', 'dti', 'earliest_cr_line', 'open_acc', 'revol_bal', 'total_acc', 'acc_now_delinq', 'GoodLoan']
len(newColNames)

18

In [117]:
loanbook_temp = loanbook_full[newColNames]

In [105]:
loanbook_full['int_rate'] = loanbook_full.int_rate.replace({'%':''}, regex=True)

In [108]:
loanbook_full['int_rate']=loanbook_full.int_rate.astype(float)

In [118]:
loanbook_full.to_csv("CleanLoanData.csv")